In [1]:
import sys
import os


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
print(sys.path[-1])

/home/cotsios/dsit/2nd-semester/ml-in-comp-bio/Assignment-2


In [2]:
from src.classifiers import pipeline
import pandas as pd

In [3]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(BASE_DIR, "data")
MODELS_DIR = os.path.join(BASE_DIR, "models")
RESULTS_DIR = os.path.join(BASE_DIR, "results")
CLASSIFIERS = [
    "LogisticRegression",
    "GaussianNB",
    "LinearDiscriminantAnalysis",
    "SVC",
    "RandomForestClassifier",
    "LGBMClassifier"
]
DF = pd.read_csv(os.path.join(DATA_DIR, "breast_cancer.csv"))
TARGET = "diagnosis"
N_ROUNDS = 10
N_OUTER_FOLDS = 5
N_INNER_FOLDS = 3
N_FEATURES_TO_SELECT = 5
N_OPTUNA_TRIALS = 50
VALIDATION_SET_FRACTION = 0.1
SEED = 42

In [4]:
val_set = pipeline(
        classifiers=CLASSIFIERS,
        df=DF,
        target=TARGET,
        models_dir=MODELS_DIR,
        results_dir=RESULTS_DIR,
        n_rounds=N_ROUNDS,
        n_outer_folds=N_OUTER_FOLDS,
        n_inner_folds=N_INNER_FOLDS,
        n_features_to_select=N_FEATURES_TO_SELECT,
        n_optuna_trials=N_OPTUNA_TRIALS,
        validation_set_fraction=VALIDATION_SET_FRACTION,
        seed=SEED
)


--- LogisticRegression ---

Running Nested Cross Validation for LogisticRegression...
Random state base: 42
Number of rounds: 10
Number of outer folds: 5
Number of inner folds: 3
Number of features to select: 5
Number of Optuna trials: 50

Round 1/10...

Outer fold 1/5...
Inner fold 3/3...
Best trial score: 0.9486965773990359
Best hyperparameters: {'C': 33.83671773036384, 'l1_ratio': 0.45276482992726585, 'max_iter': 403, 'class_weight': 'balanced'}
Outer fold 1/5 results:
Accuracy: 0.8602
Precision: 0.7805
Recall: 0.8889
Specificity: 0.8421
MCC: 0.7171
F1: 0.8312
ROC AUC: 0.8655

Outer fold 2/5...
Inner fold 3/3...
Best trial score: 0.9490789363156102
Best hyperparameters: {'C': 1593.0331037685787, 'l1_ratio': 0.4117545361946098, 'max_iter': 4891, 'class_weight': None}
Outer fold 2/5 results:
Accuracy: 0.9565
Precision: 0.9697
Recall: 0.9143
Specificity: 0.9825
MCC: 0.9077
F1: 0.9412
ROC AUC: 0.9484

Outer fold 3/5...
Inner fold 3/3...
Best trial score: 0.9328736058850428
Best hyperpa

In [5]:
val_set.to_csv(os.path.join(DATA_DIR, "validation_set.csv"), index=False)